# Outline
Text goes here

In [20]:
# Imports
import pandas as pd
from citipy import citipy
import random
import requests

# Get API key from api_keys.py
import api_keys
api_ow = api_keys.api_ow

In [4]:
# Sample API call for OpenWeather geolocation:
# http://api.openweathermap.org/geo/1.0/direct?q=CITYNAME,STATECODE,COUNTRYCODE&limit=LIMIT&appid=APIKEY

# Sample API call for OpenWeather weather data:
# units = "imperal"
#api_string = "https://api.openweathermap.org/data/2.5/weather?lat=" + LATITUDE + "&lon=" + LONGITUDE + "&appid=" + api_ow + "&units=" + units

In [23]:
# Functions

# Generates a random number between -180 and 180 degrees (0 is Greenwich, England).
def genLatitude():
    randomLat = random.uniform(-180, 180)
    return round(randomLat, 2)

# Generates a random number between -90 and 90 degrees (0 is equator)
def genLongitude():
    randomLong = random.uniform(-90, 90)
    return round(randomLong, 2)

# Send an OpenWeather API request with a set of latitude and longitude coordinates and the type of units (valid values are "standard", "metric", "imperial").
# Response will be in JSON. Extract temperature, humidity, cloudiness, wind speed.
# Return a single-row dataframe.
def getWeatherData(lat, long, units):
    api_string = "https://api.openweathermap.org/data/2.5/weather?lat=" + lat + "&lon=" + long + "&appid=" + api_ow + "&units=" + units
    result = requests.get(api_string)
    

In [19]:
# Dataframe to hold city data.
df_cities = pd.DataFrame(columns=["City Name", "Latitude", "Longitude", "Temperature (F)", "Humidity (%)", "Cloudiness (%)", "Wind speed (MPH)"])
df_cities

,City Name,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind speed (MPH)


In [24]:
# Generate 500 random sets of long/lat coordinates.
# Get the nearest city to each coordinates. If it's not in the list, add its weather data.
cities_500 = []
count = 1
while count <= 500:
    long = genLongitude()
    lat = genLatitude()
    nearest = citipy.nearest_city(lat, long)
    city = [nearest.city_name, nearest.country_code]

    # Only add the city if it's unique
    if city not in cities_500:
        cities_500.append(city)
        
        count += 1

In [ ]:
# For each city, get its temperature, humidity, cloudiness, and wind speed
# Save the data into a dataframe

In [25]:
cities_500

[['ushuaia', 'ar'],
 ['ilulissat', 'gl'],
 ['grytviken', 'gs'],
 ['tazovsky', 'ru'],
 ['luderitz', 'na'],
 ['camayenne', 'gn'],
 ['fannuj', 'ir'],
 ['corinto', 'ni'],
 ['georgetown', 'sh'],
 ['ambodifotatra', 'mg'],
 ['mount pearl', 'ca'],
 ['cargados carajos', 'mu'],
 ['longyearbyen', 'sj'],
 ['pico truncado', 'ar'],
 ["nar'yan-mar", 'ru'],
 ['constantia', 'za'],
 ['port-aux-francais', 'tf'],
 ['isafjordur', 'is'],
 ['ribeira grande', 'pt'],
 ['vorgashor', 'ru'],
 ['nova sintra', 'cv'],
 ['piracanjuba', 'br'],
 ['inebolu', 'tr'],
 ['onguday', 'ru'],
 ['edinburgh of the seven seas', 'sh'],
 ['nadym', 'ru'],
 ['znamenskoye', 'ru'],
 ['dudinka', 'ru'],
 ['iqaluit', 'ca'],
 ['hithadhoo', 'mv'],
 ['tolanaro', 'mg'],
 ['port mathurin', 'mu'],
 ['luena', 'ao'],
 ['olonkinbyen', 'sj'],
 ['nyrob', 'ru'],
 ['hamilton', 'bm'],
 ['awjilah', 'ly'],
 ["st. john's", 'ca'],
 ['vadso', 'no'],
 ['espargos', 'cv'],
 ['mount darwin', 'zw'],
 ['yagry', 'ru'],
 ['pulsano', 'it'],
 ['port elizabeth', 'za'],